In [11]:
import os

print(os.getcwd()) 
#gt_stutter = 'libristutter_result\gt_libristutter.csv'
stutter_transcription = 'libristutter_result\wav2vec_libristutter.csv'
gt_stutter = 'librispeech_result\gt_librispeech.csv'

print(os.path.exists(gt_stutter))  # This will print True if folder1 exists, otherwise False
print(os.path.exists(stutter_transcription))  # This will print True if folder2 exists, otherwise False

c:\Users\xnishikawa\OneDrive - Olin College of Engineering\Documents\GitHub\stuttered-speech-asr
True
True


In [12]:
import pandas as pd

df_gt_stutter = pd.read_csv(gt_stutter)
df_stutter_transcription = pd.read_csv(stutter_transcription)

print(df_gt_stutter.shape)
print(df_stutter_transcription.shape)

(15237, 3)
(5590, 3)


In [14]:
# Remove the first column (index 0)
df_gt_stutter_clean = df_gt_stutter.drop(df_gt_stutter.columns[0], axis=1)

# Reset the index if needed
df_gt_stutter_clean = df_gt_stutter_clean.reset_index(drop=True)
#df_gt_stutter_clean = df_gt_stutter_clean.drop('total_stutter', axis=1)
df_gt_stutter_clean['file_name'] = df_gt_stutter_clean['file_name'].str.replace('.csv', '', regex=False)

df_gt_stutter_clean.head

<bound method NDFrame.head of               file_name                                       ground_truth
0       441-128982-0000  when the little fleet of spanish vessels inclu...
1       441-128982-0001  the captain who had bartholemy and the others ...
2       441-128982-0002  for as they were all very able bodied men they...
3       441-128982-0003  and who had heard of his various exploits he t...
4       441-128982-0004  and who ought immediately to be delivered up t...
...                 ...                                                ...
15232  8098-275181-0029  his left hand was bitten through the palm and ...
15233  8098-275181-0030  these numerous bites however were followed by ...
15234  8098-275181-0031  i just determined to have it out with him cons...
15235  8098-275181-0032  i do not think that he ever cared to make a fo...
15236  8098-275181-0033  how or by what wiles that fox got the turkeys ...

[15237 rows x 2 columns]>

In [15]:
# Remove the first column (index 0)
df_stutter_transcription_clean = df_stutter_transcription.drop(df_stutter_transcription.columns[0], axis=1)

# Reset the index if needed
df_stutter_transcription_clean['file_name'] = df_stutter_transcription_clean['file_name'].str.replace('.flac', '', regex=False)

df_stutter_transcription_clean.head

<bound method NDFrame.head of             file_name                                         prediction
0     6925-80680-0006  the king made king made king made no remark bu...
1     6925-80680-0072  amer ali bowed low and answered that he had un...
2     6925-80680-0040  and as the storm had by this time passed over ...
3     6925-80680-0036  ameer ali thought that her voice sounded rathe...
4     6925-80680-0070  then the princess sent for her father and told...
...               ...                                                ...
5585  289-121652-0022  indeed as mister william h whitmore points out...
5586  289-121652-0007  are doubtless offshoots from ancient folklor s...
5587  289-121652-0004  are evidently drawn from a variety of sources ...
5588  289-121652-0011  although perreut's tals differ materially uly ...
5589  289-121652-0023  boston children at that date says mister whitm...

[5590 rows x 2 columns]>

In [17]:
import pandas as pd

# Merging dataframes based on the filename column
merged_df = pd.merge(df_gt_stutter_clean[['file_name', 'ground_truth']], df_stutter_transcription_clean[['file_name', 'prediction']], on='file_name')

# Display the merged dataframe
print(merged_df.head)


<bound method NDFrame.head of              file_name                                       ground_truth  \
0      5561-41615-0000                    chapter fifteen a would be lark   
1      5561-41615-0001  the majority however appeared to be highly del...   
2      5561-41615-0002  and as the morning advanced they became fully ...   
3      5561-41615-0003  at recess there was more grouping and whisperi...   
4      5561-41615-0004  eleanor started that paper whatever it is said...   
...                ...                                                ...   
5585  8098-275181-0029  his left hand was bitten through the palm and ...   
5586  8098-275181-0030  these numerous bites however were followed by ...   
5587  8098-275181-0031  i just determined to have it out with him cons...   
5588  8098-275181-0032  i do not think that he ever cared to make a fo...   
5589  8098-275181-0033  how or by what wiles that fox got the turkeys ...   

                                             

In [18]:
from sklearn.model_selection import train_test_split

# Splitting the merged dataframe into training and testing sets
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# Exporting the training and testing sets to separate CSV files
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

merged_df.to_csv('merged_data.csv', index=False)

In [19]:
# Validation that the data was correct
%pip install jiwer




Note: you may need to restart the kernel to use updated packages.


In [21]:
import jiwer

# Calculate WER for each pair of ground truth and prediction
ground_truth_list = merged_df['ground_truth'].tolist()
prediction_list = merged_df['prediction'].tolist()

# Calculate WER for each row
wer_results = [
    jiwer.wer(gt, pred)
    for gt, pred in zip(ground_truth_list, prediction_list)
]

# Add WER results to the dataframe
merged_df['wer'] = wer_results

# Calculate the mean WER
mean_wer = merged_df['wer'].mean()

print(f"The mean Word Error Rate is: {mean_wer:.4f}")


The mean Word Error Rate is: 0.1133
